In [7]:
%config InlineBackend.figure_formats = ['svg']
%cp -v ../coronavirus.py .
from coronavirus import *

../coronavirus.py -> ./coronavirus.py


In [9]:
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

In [10]:
def modify_template(templatefile, outputfile, mappings):
    """Create concrete *.ipynb file from template"""
    # open template
    with open(templatefile, "tr") as f_template:
        template = f_template.read()
    for key in mappings:
        template = template.replace(key, mappings[key])
    with open(outputfile, "tw") as f:
        f.write(template)
    print(f"Written file to {outputfile}")


In [11]:
def check_country_name_is_known(name):
    d = fetch_deaths()
    assert name in d.index, f"{name} is unknown. Known countries are {sorted(d.index)}"

def sanitize(name):
    """Given a country name as a string, sanitise it for use as URL and filename: 
    - get rid of spaces, commas
    
    return cleaned string.
    
    (Leave umlaute for now)
    """
    s = name.replace(" ", "-")
    s = s.replace(",", "-")
    return s
    

def create_ipynb_for_country(country):
    check_country_name_is_known(country)
    
    country = sanitize(country)
    mappings = {
        "%title%" : country,
        "%title2%" : "(data from Johns Hopkins)",
        "%country%" : country
    }
    output_file_path = f"html/{country}.ipynb"
    modify_template("template-country.ipynb", output_file_path, mappings)
    assert os.path.exists(output_file_path)
    return output_file_path


In [12]:
import subprocess
def nbconvert_ipynb2html(ipynb_path):
    command = f"cp -fv ../coronavirus.py html"
    subprocess.check_call(command, shell=True)
    
    command = f"jupyter-nbconvert {ipynb_path} --to html --execute"
    print(f"Command = {command}")
    output = subprocess.check_output(command, shell=True)
    print(f"Output is {output.decode()}")
    # compute output path
    output_path = os.path.splitext(ipynb_path)[0] + ".html"
    assert os.path.exists(output_path)
    return output_path
    

In [13]:
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

created_files = {}
country = "Italy"
#for country in ["Italy", "Germany", "US"]:
for country in countries:
    ipynb_path = create_ipynb_for_country(country)
    html_path = nbconvert_ipynb2html(os.path.join(ipynb_path))
    created_files[country] = html_path, ipynb_path

Written file to html/Afghanistan.ipynb
Command = jupyter-nbconvert html/Afghanistan.ipynb --to html --execute
Output is 
Written file to html/Albania.ipynb
Command = jupyter-nbconvert html/Albania.ipynb --to html --execute
Output is 
Written file to html/Algeria.ipynb
Command = jupyter-nbconvert html/Algeria.ipynb --to html --execute
Output is 
Written file to html/Andorra.ipynb
Command = jupyter-nbconvert html/Andorra.ipynb --to html --execute
Output is 
Written file to html/Angola.ipynb
Command = jupyter-nbconvert html/Angola.ipynb --to html --execute
Output is 
Written file to html/Antigua-and-Barbuda.ipynb
Command = jupyter-nbconvert html/Antigua-and-Barbuda.ipynb --to html --execute


CalledProcessError: Command 'jupyter-nbconvert html/Antigua-and-Barbuda.ipynb --to html --execute' returned non-zero exit status 1.

In [81]:
import os


In [86]:
os.path.splitext("html/test.ipynb")

('html/test', '.ipynb')